_________________________________________
This code is an example of how to fine tune with Lora.  Fine tuning is a way of improving the training of a model.  So instead of creating a model from scratch with the billions of dollars involved in that process, fine tuning can retrain a small portion of the model and improve results dramatically.
_________________________________________
  

In [2]:
# Step 1: Install Necessary Libraries not already installed
!pip install datasets   
!pip install evaluate
!pip install transformers
!pip install --upgrade huggingface_hub
!pip install peft==0.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 8.9 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 6.6 MB/s eta 0:00:00:00:01


In [3]:
# Step 2: Import Required Libraries
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

import huggingface_hub
import peft

import evaluate
import numpy as np
import torch
from torch.nn.functional import softmax

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [4]:
#This code block loads a pretrained model.  We use distilbert-base-uncased
  #because it is small and efficient and ignores upper or lower case but other
  #BERT models should work providing slightly better or worse results

model_chkpt = 'distilbert-base-uncased'

#define label maps.  This changes binary to text to characterize a review as positive or negative
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE":0, "POSITIVE":1}

#Load the base model before fine tuning
model = AutoModelForSequenceClassification.from_pretrained(model_chkpt, num_labels=2
                                                           , id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Step 3: Load and Prepare Dataset
#load dataset of movie reviews from IMDB
dataset = load_dataset('imdb')
dataset

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
# Step 4: Tokenize Dataset
# Tokenize the dataset using a pretrained tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_chkpt, add_prefix_space=True)

# create tokenize function
def tokenize_function(examples):
  #extract text
  text = examples["text"]

  #tokenize and truncate texte
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(text,
                                return_tensors= 'np',
                                max_length=512,
                                truncation=True)

  return tokenized_inputs

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [7]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
#This creates a metric for determining the accuracy of the model

accuracy = evaluate.load("accuracy")

# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [9]:
# define list of examples for the model to determine if the review is positive or negative


test_list = [
    "It was good",
    "Not a fan, don't recommend",
    "Better than the first one",
    "This is not worth watching once",
    "This one is a pass",
    "Loved it, highly recommend!",
    "Terrible plot, skip it.",
    "Surprisingly entertaining!",
    "A dull and predictable mess.",
    "Decent, but nothing special.",
    "Fresh and fun to watch!",
    "Confusing from start to end.",
    "Better than I expected.",
    "Lacked heart and emotion.",
    "Perfect for a lazy afternoon.",
    "Not great, but not awful.",
    "A total waste of two hours.",
    "Visually stunning, great vibes!",
    "Nothing memorable here.",
    "A sequel that actually works!"
]

print("Untrained model predictions plus confidence of prediction:")
print("_____________________________")

for text in test_list:
    inputs=tokenizer.encode(text, return_tensors='pt')

    # Ensure the model is in evaluation mode
    model.eval()

    with torch.no_grad():
        logits = model(inputs).logits

    probabilities = softmax(logits, dim=1)  # Convert logits to probabilities
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0][predicted_class].item() * 100

    print(f"{text} - {id2label[predicted_class]} - {confidence:.2f}%")



Untrained model predictions plus confidence of prediction:
_____________________________
It was good - NEGATIVE - 50.50%
Not a fan, don't recommend - NEGATIVE - 51.08%
Better than the first one - NEGATIVE - 51.16%
This is not worth watching once - NEGATIVE - 51.22%
This one is a pass - NEGATIVE - 50.44%
Loved it, highly recommend! - NEGATIVE - 50.44%
Terrible plot, skip it. - NEGATIVE - 51.95%
Surprisingly entertaining! - NEGATIVE - 51.36%
A dull and predictable mess. - NEGATIVE - 51.25%
Decent, but nothing special. - NEGATIVE - 50.86%
Fresh and fun to watch! - NEGATIVE - 52.27%
Confusing from start to end. - NEGATIVE - 51.73%
Better than I expected. - NEGATIVE - 51.13%
Lacked heart and emotion. - NEGATIVE - 50.59%
Perfect for a lazy afternoon. - NEGATIVE - 51.28%
Not great, but not awful. - NEGATIVE - 50.92%
A total waste of two hours. - NEGATIVE - 51.57%
Visually stunning, great vibes! - NEGATIVE - 50.82%
Nothing memorable here. - NEGATIVE - 51.18%
A sequel that actually works! - NEG

_________________________________________
Here are the results generated

Untrained model predictions:
_____________________________
It was good - POSITIVE
Not a fan, don't recommend - POSITIVE
Better than the first one - POSITIVE
This is not worth watching once - POSITIVE
This one is a pass - POSITIVE
Loved it, highly recommend! - POSITIVE
Terrible plot, skip it. - POSITIVE
Surprisingly entertaining! - POSITIVE
A dull and predictable mess. - POSITIVE
Decent, but nothing special. - POSITIVE
Fresh and fun to watch! - POSITIVE
Confusing from start to end. - POSITIVE
Better than I expected. - POSITIVE
Lacked heart and emotion. - POSITIVE
Perfect for a lazy afternoon. - POSITIVE
Not great, but not awful. - POSITIVE
A total waste of two hours. - POSITIVE
Visually stunning, great vibes! - POSITIVE
Nothing memorable here. - POSITIVE
A sequel that actually works! - POSITIVE
_________________________________________

In [10]:
#This is the code for finetuning the model.  YOu can change these parameters
  #to balance speed with accuracy.  The r and lora_alpha are the two parameters
  #that will affect this the most.  Note the trainable% output in the next code set.
  #The higher the trainable%, the better the accuracy but the slower the model.

peft_config = LoraConfig(
    task_type="SEQ_CLS", #Indicates sequence classification
    r=4, #rank of trainable weight matrix
    lora_alpha=32,  #Equivilent to a learning rate
    lora_dropout=0.01, #probability of dropout
    target_modules=["q_lin"] # use lora on query layer
)



In [11]:
#Load model and output the trainable parameters vs total parameters

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


_________________________________________
Here are the results.  

*trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307*

Note the trainable percentage.  This indicates that the fine tuning is changing less than 1% of the model parameters.  This is much cheaper and faster than retraining 100% of the model.
________________________________________  

In [12]:
#These sets the hyperparameters for the finetuned model.

#hyperparameters
lr= 3e-5
batch_size=4
num_epochs=10

#training arguements

training_args = TrainingArguments(
    output_dir= model_chkpt + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

In [13]:
#This trains the model on just the portion set to fine tuning.
  #This will be very slow and perhaps unusable without a GPU.

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    #processing_class=tokenizer,   #Trainer args has changed.  Older versions use tokenizer
                                    #newer versions use processing_class
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

#Train model
trainer.train()
###

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.270800,0.268274,{'accuracy': 0.8978}
2,0.270100,0.255012,{'accuracy': 0.90924}
3,0.253900,0.249668,{'accuracy': 0.9138}
4,0.243500,0.251947,{'accuracy': 0.91576}
5,0.245900,0.248653,{'accuracy': 0.91668}
6,0.227800,0.243733,{'accuracy': 0.91756}
7,0.229300,0.254362,{'accuracy': 0.91876}
8,0.216700,0.253325,{'accuracy': 0.91836}
9,0.233400,0.250400,{'accuracy': 0.91868}
10,0.233000,0.251609,{'accuracy': 0.91916}


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=31250, training_loss=0.2535447918701172, metrics={'train_runtime': 8830.1166, 'train_samples_per_second': 28.312, 'train_steps_per_second': 3.539, 'total_flos': 3.1645323781065024e+16, 'train_loss': 0.2535447918701172, 'epoch': 10.0})

In [14]:
import os

fine_tuned_model_path = "distilbert-base-uncased-lora-text-classification"
print(os.listdir(fine_tuned_model_path))


if os.path.exists(fine_tuned_model_path):
    print(f"The directory '{fine_tuned_model_path}' exists.")
else:
    print(f"The directory '{fine_tuned_model_path}' does NOT exist.")



['checkpoint-31250', 'checkpoint-15625', 'checkpoint-28125', 'checkpoint-12500', 'checkpoint-25000', 'checkpoint-6250', 'checkpoint-21875', 'checkpoint-3125', 'checkpoint-9375', 'checkpoint-18750']
The directory 'distilbert-base-uncased-lora-text-classification' exists.


In [15]:
trainer.save_model("distilbert-base-uncased-lora-text-classification")


In [16]:
#This tests the new fine tuned model on the example from earlier to see if there
  #is an improvement.

# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained(fine_tuned_model_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_chkpt, add_prefix_space=True)

# Define the list of examples
test_list = [
    "It was good",
    "Not a fan, don't recommend",
    "Better than the first one",
    "This is not worth watching once",
    "This one is a pass",
    "Loved it, highly recommend!",
    "Terrible plot, skip it.",
    "Surprisingly entertaining!",
    "A dull and predictable mess.",
    "Decent, but nothing special.",
    "Fresh and fun to watch!",
    "Confusing from start to end.",
    "Better than I expected.",
    "Lacked heart and emotion.",
    "Perfect for a lazy afternoon.",
    "Not great, but not awful.",
    "A total waste of two hours.",
    "Visually stunning, great vibes!",
    "Nothing memorable here.",
    "A sequel that actually works!"
]

print("Trained model predictions:")
print("_____________________________")

# Make predictions using the trained model
for text in test_list:
    inputs = tokenizer.encode(text, return_tensors='pt')

    # Ensure the model is in evaluation mode
    model.eval()

    with torch.no_grad():
        logits = model(inputs).logits

    probabilities = softmax(logits, dim=1)  # Convert logits to probabilities
    predicted_class = torch.argmax(probabilities, dim=1).item()
    confidence = probabilities[0][predicted_class].item() * 100

    print(f"{text} - {id2label[predicted_class]} - {confidence:.2f}%")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Trained model predictions:
_____________________________
It was good - POSITIVE - 90.81%
Not a fan, don't recommend - NEGATIVE - 93.56%
Better than the first one - POSITIVE - 92.05%
This is not worth watching once - POSITIVE - 60.50%
This one is a pass - POSITIVE - 83.91%
Loved it, highly recommend! - POSITIVE - 98.26%
Terrible plot, skip it. - NEGATIVE - 99.56%
Surprisingly entertaining! - POSITIVE - 93.83%
A dull and predictable mess. - NEGATIVE - 99.36%
Decent, but nothing special. - NEGATIVE - 89.03%
Fresh and fun to watch! - POSITIVE - 99.79%
Confusing from start to end. - POSITIVE - 52.08%
Better than I expected. - POSITIVE - 95.88%
Lacked heart and emotion. - NEGATIVE - 62.29%
Perfect for a lazy afternoon. - POSITIVE - 82.45%
Not great, but not awful. - POSITIVE - 77.93%
A total waste of two hours. - NEGATIVE - 97.45%
Visually stunning, great vibes! - POSITIVE - 99.17%
Nothing memorable here. - NEGATIVE - 98.00%
A sequel that actually works! - POSITIVE - 73.38%


Trained model predictions:
*-Incorrect
$-Neutral Statement
_____________________________
It was good - POSITIVE - 90.81%

Not a fan, don't recommend - NEGATIVE - 93.56%

Better than the first one - POSITIVE - 92.05%

*This is not worth watching once - POSITIVE - 60.50%

*This one is a pass - POSITIVE - 83.91%

Loved it, highly recommend! - POSITIVE - 98.26%

Terrible plot, skip it. - NEGATIVE - 99.56%

Surprisingly entertaining! - POSITIVE - 93.83%

A dull and predictable mess. - NEGATIVE - 99.36%

$Decent, but nothing special. - NEGATIVE - 89.03%

Fresh and fun to watch! - POSITIVE - 99.79%

*Confusing from start to end. - POSITIVE - 52.08%

Better than I expected. - POSITIVE - 95.88%

Lacked heart and emotion. - NEGATIVE - 62.29%

Perfect for a lazy afternoon. - POSITIVE - 82.45%

$Not great, but not awful. - POSITIVE - 77.93%

A total waste of two hours. - NEGATIVE - 97.45%

Visually stunning, great vibes! - POSITIVE - 99.17%

Nothing memorable here. - NEGATIVE - 98.00%

A sequel that actually works! - POSITIVE - 73.38%

_____________________________________________________

This is much better!  We have 15 correct, 3 incorrect and 2 relatively neutral statements.  That's a big improvement.  Looking at the confidence level, we can make a couple of observations.  All of the mistakes and neutral statements are below 90%.  Unfortunately, we have a few correct that are below 90%.

Ideas for improvement:
-Use a different model like Bert
-Change the hyperparameters of the fine tuning like the learning rate or the number of epochs